In [1]:
from sklearn.linear_model import LinearRegression, LogisticRegression
import numpy as np
import requests

import aiohttp
import asyncio

import nest_asyncio
nest_asyncio.apply()

base_url = 'http://127.0.0.1:8000'

Код вызова последовательного вызова обучения как минимум двух  различных моделей с таким набором данных и параметрами, чтобы обучение одной модели длилось не менее 60 секунд

In [2]:
X = [[np.random.randint(0, 100) for i in range(10**5)] for i in range(10**2)]
y = [np.random.randint(0, 100) for i in range(10**2)]

In [3]:
X_logreg = [[np.random.randint(0, 100) for i in range(10**5)] for i in range(10**2)]
y_logreg = [np.random.randint(0, 2) for i in range(10**2)]

In [4]:
data = [{
    "X" : X,
    "config" : {
      "hyperparameters": {
        "fit_intercept": True
      },
      "id": "linear_123",
      "ml_model_type": "linear"
    },
    "y" : y
},
{
    "X" : X_logreg,
    "config" : {
      "hyperparameters": {
        "fit_intercept": True
      },
      "id": "logreg_123",
      "ml_model_type": "logistic"
    },
    "y" : y_logreg
}
]

response = requests.post(f'{base_url}/api/v1/models/fit', json=data)

print(response.status_code)
print(response.json())

201
[{'messeage': 'Model linear_123 trained and saved'}, {'messeage': 'Model logreg_123 trained and saved'}]


загрузка моделей

In [ ]:
data = {"id" : "linear_123"}
response = requests.post(f'{base_url}/api/v1/models/load', json=data)

print(response.status_code)
print(response.json())

data = {"id" : "logreg_123"}
response = requests.post(f'{base_url}/api/v1/models/load', json=data)

print(response.status_code)
print(response.json())

200
[{'message': 'Model linear_123 loaded'}]
200
[{'message': 'Model logreg_123 loaded'}]


Асинхронный вызов нескольких предсказаний

In [7]:
async def fetch(session, url, data):
    async with session.post(url, json=data) as response:
        return await response.json()

async def main():
    urls = [
        f"{base_url}/api/v1/models/predict",
        f"{base_url}/api/v1/models/predict"
    ]

    datas = [{"id" : "linear_123", "X" : X}, {"id" : "logreg_123", "X" : X_logreg}]
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        
        for url, data in zip(urls, datas):
            task = asyncio.create_task(fetch(session, url, data))
            tasks.append(task)
            
        result = await asyncio.gather(*tasks)
        print(result)

asyncio.run(main())

[[{'predictions': [35.0, 40.999999999999936, 12.99999999999988, 10.999999999999893, 65.00000000000014, 41.00000000000002, 23.99999999999986, 95.00000000000009, 85.99999999999989, 2.000000000000014, 92.99999999999983, 29.999999999999947, 19.00000000000001, 49.99999999999999, 74.00000000000011, 12.999999999999964, 34.00000000000005, 94.00000000000011, 52.00000000000003, 47.0, 85.00000000000009, 76.00000000000009, 65.00000000000001, 78.00000000000009, 99.00000000000006, 73.99999999999997, 22.99999999999988, 5.000000000000121, 52.00000000000006, 54.000000000000185, 34.00000000000005, 27.999999999999932, 63.00000000000003, 48.00000000000004, 21.000000000000163, 18.000000000000025, 31.0, 31.00000000000011, 14.999999999999957, 6.000000000000014, 54.00000000000001, 7.000000000000085, 14.999999999999766, 12.999999999999844, 20.000000000000004, 90.9999999999998, 20.99999999999988, 25.99999999999992, 37.9999999999999, 31.99999999999999, 29.99999999999998, 31.999999999999993, 89.0, 49.999999999999

список обученных моделей

In [8]:
response = requests.get(f'{base_url}/api/v1/models/list_models')
response.json()

[{'id': 'linear_123', 'type': 'linear'},
 {'id': 'logreg_123', 'type': 'logistic'}]

выгрузка моделей 

In [9]:
response = requests.post(f'{base_url}/api/v1/models/unload')
print(response.json())
print(response)

[{'message': 'Model linear_123 unloaded'}, {'message': 'Model logreg_123 unloaded'}]
<Response [200]>


удаление всех обученных моделей

In [10]:
response = requests.delete(f'{base_url}/api/v1/models/remove_all')
response.json()

[{'message': 'Model linear_123 removed'},
 {'message': 'Model logreg_123 removed'}]

снова посмотрим список обученных моделей

In [11]:
response = requests.get(f'{base_url}/api/v1/models/list_models')
response.json()

[]

# демонстрация обработки исключений

обучим модель с неаправильным типом

In [48]:
data = [{
    "X" : X,
    "config" : {
      "hyperparameters": {
        "fit_intercept": True
      },
      "id": "linear_123",
      "ml_model_type": "llllllllll"
    },
    "y" : y
}
]

response = requests.post(f'{base_url}/api/v1/models/fit', json=data)

print(response.json())

{'detail': 'Несуществующий тип модели: llllllllll'}


попробуем обучить с неправильными параметрами модели

In [16]:
data = [{
    "X" : X,
    "config" : {
      "hyperparameters": {
        "fit_fit_fit": True
      },
      "id": "linear_123",
      "ml_model_type": "linear"
    },
    "y" : y
}
]

response = requests.post(f'{base_url}/api/v1/models/fit', json=data)

print(response.json())

{'detail': 'Некорректные параметры модели'}


снова обучим модель правильно

In [49]:
data = [{
    "X" : X,
    "config" : {
      "hyperparameters": {
        "fit_intercept": True
      },
      "id": "linear_123",
      "ml_model_type": "linear"
    },
    "y" : y
}
]

response = requests.post(f'{base_url}/api/v1/models/fit', json=data)

print(response.status_code)
print(response.json())

201
[{'messeage': 'Model linear_123 trained and saved'}]


попытка загрузить с несуществующим id

In [17]:
data = {"id" : "linear_13"}
response = requests.post(f'{base_url}/api/v1/models/load', json=data)

print(response.json())

{'detail': 'Несуществующий id модели'}


загрузим правильно, что сдеалть predict

In [50]:
data = {"id" : "linear_123"}
response = requests.post(f'{base_url}/api/v1/models/load', json=data)

print(response.json())

[{'message': 'Model linear_123 loaded'}]


Тут отрабатывают несколько raise: на id и размерность данных, продеманстрирую только на размерности данных, так как проверка id идентична предыдущим примерам

In [37]:
async def fetch(session, url, data):
    async with session.post(url, json=data) as response:
        return await response.json()

async def main():
    urls = [
        f"{base_url}/api/v1/models/predict"
    ]

    datas = [{"id" : "linear_123", "X" : [[1,2,3]]}]
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        
        for url, data in zip(urls, datas):
            task = asyncio.create_task(fetch(session, url, data))
            tasks.append(task)
            
        result = await asyncio.gather(*tasks)
        print(result)

asyncio.run(main())

[{'detail': 'Некорректная размерность данных'}]


список обученных моделей до удаления

In [39]:
response = requests.get(f'{base_url}/api/v1/models/list_models')
response.json()

[{'id': 'linear_123', 'type': 'linear'}]

попытка удалить по неверному id

In [47]:
id_='linear_1233'
response = requests.delete(f'{base_url}/api/v1/models/remove/{id_}', data=id_)
response.json()

{'detail': 'Несуществующий id модели'}

In [52]:
id_='linear_123'
response = requests.delete(f'{base_url}/api/v1/models/remove/{id_}', data=id_)
response.json()

[{'message': 'Model linear_123 removed'}]